In [1]:
#-*- encoding: utf-8 -*-

import os
import numpy as np
import imp

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from dataset import MNIST, Feature
# import model
# imp.reload(model)   # 不这样reload，调试的时候修改引用的py文件是没有作用的
# from model import Net, LossFunc, Optim


In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size = 9
hidden_size = 5
hidden2_size = 5
num_classes = 4
num_epochs = 200
batch_size = 1000
learning_rate = 0.001

In [3]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, hidden2_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        # self.fc2 = nn.Linear(hidden_size, hidden2_size)
        # self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        # out = self.fc2(out)
        # out = self.relu(out)
        out = self.fc3(out)
        return out

In [4]:
model = NeuralNet(input_size, hidden_size, hidden2_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
############### data pre-processing ###############

train_set = MNIST('mnist', 'train', (10, 10))
test_set = MNIST('mnist', 't10k', (10, 10))

train_feature = Feature(train_set.data, kernel_size=(4,4), stride=(3,3))
train_v, train_label = train_feature.calc_feature_vector()
# train_fv, train_label = train_feature.encoding(train_v, is_round=True)
# train_feature.hist(save_path='log/train_hist.png', data=train_fv)
# input_train_data = train_feature.cut_into_batch(batch_size=1000, vector=train_fv, labels=train_label)
train_fv, train_label = train_feature.extract_num_class([0, 1, 4, 7], train_v, train_label)
train_fv = train_fv.reshape(-1, 9)
# train_fv = train_feature._data['images'].reshape(-1, 100)
# train_label = train_feature._data['labels']

test_feature = Feature(test_set.data, kernel_size=(4,4), stride=(3,3))
test_v, test_label = test_feature.calc_feature_vector()
# test_fv, test_label = test_feature.encoding(test_v, is_round=True)
# test_feature.hist(save_path='log/test_hist.png', data=test_fv)
# input_test_data = test_feature.cut_into_batch(batch_size=1000, vector=test_fv, labels=test_label)
test_fv, test_label = test_feature.extract_num_class([0, 1, 4, 7], test_v, test_label)
test_fv = test_fv.reshape(-1, 9)
# test_fv = test_feature._data['images'].reshape(-1, 100)
# test_label = test_feature._data['labels']

In [6]:
############## 创建pytoch使用的数据集 ##############

class MyDataset(Dataset): #创建自己的类：MyDataset,这个类是继承的torch.utils.data.Dataset
    def __init__(self, input, labels): #初始化一些需要传入的参数
        self.input = input
        self.labels = labels
 
    def __getitem__(self, index):
        return self.input[index], self.labels[index]
 
    def __len__(self): #这个函数也必须要写，它返回的是数据集的长度，也就是多少张图片，要和loader的长度作区分
        return self.labels.shape[0]


In [7]:
train_data = MyDataset(train_fv, train_label)
test_data = MyDataset(test_fv, test_label)

#然后就是调用DataLoader和刚刚创建的数据集，来创建dataloader，这里提一句，loader的长度是有多少个batch，所以和batch_size有关
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size)

In [8]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.float().to(device)
        labels = labels.long().to(device)

        # Forward pass
        outputs = model(images)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

42.4731,  92.3064],
        ...,
        [291.8690, 168.3273,   0.0000, 437.3553,   0.0000],
        [166.1488, 189.1833,  43.5988, 321.5572,  96.5184],
        [223.3181, 208.2751,  45.5937, 374.0827,   0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[208.9508,   3.6280, 264.7027, 178.7487, 140.0381],
        [294.0529, 192.8374,  42.5436, 437.2769,   0.0000],
        [285.2379, 202.7866, 257.6888, 410.6824, 214.2082],
        ...,
        [447.5952, 514.2845, 450.5875, 801.5633,   9.4979],
        [186.0947, 149.3399, 325.3672, 268.5378,  38.8889],
        [314.5437, 236.3480, 183.8544, 493.2284,  92.2422]],
       grad_fn=<ReluBackward0>)
tensor([[173.2906, 113.4692, 261.4665, 239.2951,  89.9574],
        [145.9613,  97.5169, 209.2927, 202.2225, 102.1147],
        [201.4291, 102.0430, 146.8692, 272.5321, 157.9138],
        ...,
        [251.4661, 216.9987,  77.8828, 373.5628,   0.0000],
        [433.8625, 151.9418, 116.4420, 524.2882, 281.6606],
        [155.3663, 413.1805, 114.85

KeyboardInterrupt: 

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.float().to(device)
        labels = labels.long().to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


In [ ]:
parm={}
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())
    parm[name]=parameters.detach().numpy()

fc1_weight = parm['fc1.weight']
fc1_bias = parm['fc1.bias']
# print(fc1_weight)
# print(fc1_bias)

for i in range(fc1_bias.shape[0]):
    bias = fc1_bias[i]
    weight = fc1_weight[i]
    print('Threshold...')
    print(bias / weight)